In [4]:
import numpy as np
from math import gcd

def mod_inverse(a, m):
    # Find the modular inverse of a under modulo m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def generate_key_matrix(key):
    # Convert key into a 2x2 matrix
    key_matrix = []
    for char in key:
        key_matrix.append(ord(char) - ord('A'))  # Convert letter to number (A=0, B=1, ...)
    matrix = np.array(key_matrix).reshape(2, 2)

    # Validate determinant
    det = int(np.round(np.linalg.det(matrix)))
    if gcd(det, 26) != 1:
        raise ValueError("Key matrix is not invertible under modulo 26. Choose a different key.")
    
    return matrix

def hill_encrypt(plaintext, key_matrix):
    if len(plaintext) % 2 != 0:
        plaintext += 'X'  # Pad plaintext if necessary

    plaintext_matrix = []
    for char in plaintext:
        plaintext_matrix.append(ord(char) - ord('A'))

    # Convert plaintext into a matrix of 2x1 pairs
    plaintext_matrix = np.array(plaintext_matrix).reshape(-1, 2).T

    # Perform matrix multiplication and mod 26
    ciphertext_matrix = np.dot(key_matrix, plaintext_matrix) % 26

    # Convert back to text
    ciphertext = ''.join(chr(num + ord('A')) for num in ciphertext_matrix.T.flatten())
    return ciphertext

def hill_decrypt(ciphertext, key_matrix):
    # Find determinant and modular inverse of the determinant under modulo 26
    det = int(np.round(np.linalg.det(key_matrix)))
    det_mod_inverse = mod_inverse(det % 26, 26)

    if det_mod_inverse is None:
        raise ValueError("Key matrix is not invertible under modulo 26.")

    # Calculate the adjugate matrix and inverse key matrix
    adjugate = np.round(det * np.linalg.inv(key_matrix)).astype(int) % 26
    inverse_key_matrix = (det_mod_inverse * adjugate) % 26

    ciphertext_matrix = []
    for char in ciphertext:
        ciphertext_matrix.append(ord(char) - ord('A'))

    # Convert ciphertext into a matrix of 2x1 pairs
    ciphertext_matrix = np.array(ciphertext_matrix).reshape(-1, 2).T

    # Perform matrix multiplication and mod 26
    plaintext_matrix = np.dot(inverse_key_matrix, ciphertext_matrix) % 26

    # Convert back to text
    plaintext = ''.join(chr(num + ord('A')) for num in plaintext_matrix.T.flatten())
    return plaintext

# Example Usage
if __name__ == "__main__":
    key = "GYBN"  # Key for generating a matrix
    plaintext = "HELLO"

    try:
        # Use the first 4 characters to create a 2x2 matrix
        key_matrix = generate_key_matrix(key[:4])

        print("Key Matrix:")
        print(key_matrix)

        # Encrypt the plaintext
        ciphertext = hill_encrypt(plaintext, key_matrix)
        print("Ciphertext:", ciphertext)

        # Decrypt the ciphertext
        decrypted_text = hill_decrypt(ciphertext, key_matrix)
        print("Decrypted Text:", decrypted_text)

    except ValueError as e:
        print(e)


Key matrix is not invertible under modulo 26. Choose a different key.
